In [1]:
#GPU number
GPU = 1

num_workers = 4

#Random Seed
seed = 19

#Target set to 'p' since we're predicting the p-factor. 
target_name = 'p'

smoothing = True
gsr = False
censor = False

In [2]:
#Pre-traned SGNN path
prt_dir = "/users/hjd/IG_my_study/SNUH/data/CFA_factors_results/pretrain_model/SparsityGrid/p/Hsp:[0.95,0.3,0.3]_Maxb:[0.01, 0.05, 0.02]_Betalr:[0.001, 0.005, 0.002]_LR:[5e-05]_Act:[elu]_Opt:[nag]_DO:[0.9,0.9,0.0]_lambda:[0.01]_seed[42]/"
print(prt_dir)

/users/hjd/IG_my_study/SNUH/data/CFA_factors_results/pretrain_model/SparsityGrid/p/Hsp:[0.95,0.3,0.3]_Maxb:[0.01, 0.05, 0.02]_Betalr:[0.001, 0.005, 0.002]_LR:[5e-05]_Act:[elu]_Opt:[nag]_DO:[0.9,0.9,0.0]_lambda:[0.01]_seed[42]/


In [3]:
import sklearn, torch
sklearn.__version__, torch.__version__

('0.24.1', '1.10.1+cu102')

In [4]:
#Index for hyperparameter selection.
#Hyperparameter sets were selected with Paramgrid, then one specific hyperparameter was chosen based on this index.
temp_sel_idx = 0 

#Number of cross-validation. 5 as default
n_cv = 20

## 1. Parameter setting

In [5]:
import numpy as np
sel_cv_idx = 0
jump_val = n_cv
outer_cv_part = np.arange(sel_cv_idx * jump_val, sel_cv_idx * jump_val + jump_val)
print("Total Fold: {}".format(outer_cv_part))

select_fold = [0,1] # #has to be list format
print("Selected Fold: {}".format(select_fold))

Total Fold: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Selected Fold: [0, 1]


In [6]:
from sklearn.model_selection import ParameterGrid

act_func_name = "elu"
optimizer_name = "nag"

#Number of nodes in NN. 1024 as default
h1_cand = [1024]
h2_cand = [1024]

#Dropout rate
dropout_h1_cand = [0.3]
dropout_h2_cand = [0.3]

#Batch size / Learning rate / LR patience for scheduling / LR scaling ratio / Epochs / Freezing epochs
batch_size_cand = [2]
lr_cand = [1e-05]
lr_patience_cand = [5]
lr_factor_cand = [0.5]
epochs_cand = [50]

#Hoyer's sparsity candidates
hsp_h1_cand = [0.95]
hsp_h2_cand = [0.3]

#L2 norm
l2_param_cand = [2e-01]

#Pretraining & layer freezing option
pretrain_cand = [True]
trainable_ext_cand = [False]
trainable_prd_cand = [True]

param_cand = {
    "h1": h1_cand, "h2": h2_cand,  
    "dropout_h1": dropout_h1_cand, "dropout_h2": dropout_h2_cand,
    "batch_size": batch_size_cand, "lr": lr_cand, "epochs": epochs_cand, 
    "lr_patience": lr_patience_cand, "lr_factor": lr_factor_cand,
    "hsp_h1": hsp_h1_cand, "hsp_h2": hsp_h2_cand, "l2_param": l2_param_cand,
    "pretrain": pretrain_cand, 
    "freeze_ext": trainable_ext_cand, "freeze_prd": trainable_prd_cand
}

In [7]:
import gc
import itertools
import math
import os
import pickle
import random
import time
from datetime import datetime as dt
from decimal import Decimal

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from pytz import timezone
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold

from PIL import Image
import imageio

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Function, Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau,CosineAnnealingWarmRestarts
from torch.optim.swa_utils import SWALR, AveragedModel
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms, utils

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU)

In [11]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [12]:
def seed_everything(seed=seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(seed)

In [13]:
nowtime = dt.now(timezone("Asia/Seoul")); year = str(nowtime.year)[2:]
month = '0{}'.format(nowtime.month) if nowtime.month < 10 else str(nowtime.month)
day = '0{}'.format(nowtime.day) if nowtime.day < 10 else str(nowtime.day)
hour = '0{}'.format(nowtime.hour) if nowtime.hour < 10 else str(nowtime.hour)
minute = '0{}'.format(nowtime.minute) if nowtime.minute < 10 else str(nowtime.minute)
sec = '0{}'.format(nowtime.second) if nowtime.second < 10 else str(nowtime.second)
msec = str(nowtime.microsecond)[:2]

## 2. Input / Target setting & Fold division

In [14]:
X =np.load(f"/data4/SNU/data/snuh_fc_temp_hjd/RSFC_Smoothing[{smoothing}]_GSR[{gsr}]_Censor[{censor}].npz")["X"]
print(X.max(),X.min())
X.shape

3.826154 -1.5008454


(125, 61776)

In [15]:
y_df = pd.read_csv("/data4/SNU/data/demo_for_rsfc_df.csv", index_col=0)
cfa_fs = np.load("/data4/SNU/data/SNUH_CFA_5factor.npz")[target_name]
y_df['fs'] = cfa_fs

# #Exclude outlier
# exclude_sbj = y_df.loc[(y_df['fs']>1.5)&(y_df['group']=='CON'),'new_id'].tolist()[0]
# exclude_sbj_idx = np.where(y_df['new_id'] == exclude_sbj)[0][0]
# y_df = y_df.loc[y_df['new_id']!=exclude_sbj,:]
# X = np.delete(X,exclude_sbj_idx,0)

y = y_df['fs'].values.reshape(-1, 1)

print(X.shape, y.shape)

(125, 61776) (125, 1)


In [16]:
from sklearn.model_selection import KFold, ShuffleSplit

outer_n_splits = n_cv

outer_train_folds_idx = []
outer_test_folds_idx = []

outer_skf = ShuffleSplit(
    n_splits=outer_n_splits, test_size=0.20, random_state=seed)

for n_outer, (outer_train_idx, outer_test_idx) in enumerate(
    outer_skf.split(X, y)):
    outer_train_folds_idx.append(outer_train_idx)
    outer_test_folds_idx.append(outer_test_idx)
    
    X_outer_train, y_outer_train = X[outer_train_idx], y[outer_train_idx]


len(outer_train_folds_idx),len(outer_train_folds_idx[0]), len(outer_test_folds_idx[0])

(20, 100, 25)

## 3. Real-time visualization code

In [17]:
num_ROIs = tot_rois = 352
threshold = 1.96

# Preparing draw feature map
parcels = pd.read_excel("/users/hjw/data/ABCD/Parcels/Parcels.xlsx", engine="openpyxl")
networks = list(parcels["Community"]) + 19 * ["Subcortex"]

networks_df = pd.DataFrame(networks, columns=["network"])
networks_df[networks_df["network"] == "Auditory"] = "AUD"
networks_df[networks_df["network"] == "Visual"] = "VIS"
networks_df[networks_df["network"] == "VentralAttn"] = "VAN"
networks_df[networks_df["network"] == "Subcortex"] = "SCN"
networks_df[networks_df["network"] == "Salience"] = "SAL"
networks_df[networks_df["network"] == "SMmouth"] = "SMM"
networks_df[networks_df["network"] == "SMhand"] = "SMH"
networks_df[networks_df["network"] == "RetrosplenialTemporal"] = "RSP"
networks_df[networks_df["network"] == "None"] = "NONE"
networks_df[networks_df["network"] == "FrontoParietal"] = "FPN"
networks_df[networks_df["network"] == "DorsalAttn"] = "DAN"
networks_df[networks_df["network"] == "Default"] = "DMN"
networks_df[networks_df["network"] == "CinguloParietal"] = "CPAR"
networks_df[networks_df["network"] == "CinguloOperc"] = "CON"

networks = np.array(networks_df["network"]).astype("str")
network_label = np.unique(networks, return_index=True)[0].astype("str")
orig_network_path = "/users/hjw/data/ABCD/npz_files/Gordon_network_labels.npz"
orig_networks = np.load(orig_network_path)["networks"]
print(orig_networks.shape)

# Set order of network label 
new_orig_network_idx_order = [
    'AUD', 'DAN', 'CPAR', 'NONE', 
    'SAL', 'VIS', 'RSP', 'DMN',
    'SMM', 'CON', 'SCN', 
    'SMH', 'VAN', 'FPN'
]

sorted_order = sorted(new_orig_network_idx_order)
sorted_order.remove("NONE")
sorted_order.append("NONE")
new_orig_network_idx_order = sorted_order
print(new_orig_network_idx_order)

new_orig_network_order = {
    key:value for (key, value) in 
    zip(new_orig_network_idx_order, (np.arange(len(new_orig_network_idx_order))))
}


ref_orig_wfm = np.zeros((tot_rois, tot_rois))
print(ref_orig_wfm.shape)
ref_orig_wfm = pd.DataFrame(ref_orig_wfm, index=orig_networks, columns=orig_networks)
ref_orig_wfm = ref_orig_wfm.sort_index(key=lambda x: x.map(new_orig_network_order), 
                                       axis=0)
ref_orig_wfm = ref_orig_wfm.sort_index(key=lambda x: x.map(new_orig_network_order), 
                                       axis=1)    

network_unq = new_orig_network_idx_order
sorted_networks = np.array(ref_orig_wfm.columns, dtype=np.str)


sorted_networks_df = pd.DataFrame(np.unique(sorted_networks, return_index=True)).T
sorted_networks_df.columns = ["networks", "n"]
sorted_networks_df = sorted_networks_df.sort_values(
    by="networks", key=lambda x: x.map(new_orig_network_order)
)

start_network_idx = np.array(sorted_networks_df.n)
next_network_idx = np.hstack((start_network_idx[1:], 352))

network_mid_idx = np.array((start_network_idx + next_network_idx) / 2, dtype=np.int)

(352,)
['AUD', 'CON', 'CPAR', 'DAN', 'DMN', 'FPN', 'RSP', 'SAL', 'SCN', 'SMH', 'SMM', 'VAN', 'VIS', 'NONE']
(352, 352)


In [18]:
def make_wfm(vec):
    wfm = np.zeros((tot_rois, tot_rois))
    iu_non_di_idx = np.mask_indices(tot_rois, np.triu, 1)
    wfm[iu_non_di_idx] = vec
    il_idx = np.tril_indices(tot_rois, -1)
    wfm[il_idx] = wfm.T[il_idx]
    wfm_df = pd.DataFrame(wfm, index=orig_networks, columns=orig_networks)
    wfm_df = wfm_df.sort_index(key=lambda x: x.map(new_orig_network_order), axis=0)
    wfm_df = wfm_df.sort_index(key=lambda x: x.map(new_orig_network_order), axis=1)
    
    return wfm_df

In [19]:
#input으로 model, epoch받아서 wfm 저장시키기

def visualize_wfm(trained_model,epoch,epoch_gap = 10,threshold=False,mode = 'sum'):
    
    if epoch%epoch_gap !=0:
        return
    
    wfm_save_dir = outer_save_dir+'/WFM/'
    if not os.path.isdir(wfm_save_dir):
        os.mkdir(wfm_save_dir)
    
    w_ext = []
    w_reg = []
    w_dsc = []

    ext_hidden = trained_model.ext_1.weight.shape[0]
    prd_hidden = trained_model.prd_1.weight.shape[0]

    w = []

    for name,params in trained_model.named_parameters():
        if "weight" in name and "bn" not in name:
            if "ext" in name or "prd" in name:
                w.append(params)

    w_ext_1 = w[0].detach().cpu().numpy().T
    w_reg_1 = w[1].detach().cpu().numpy().T
    w_reg_2 = w[2].detach().cpu().numpy().T

    temp_w_ext = w_ext_1
    temp_w_reg = np.matmul(np.matmul(temp_w_ext, w_reg_1), w_reg_2)

    w_ext.append(temp_w_ext) #ext x pred1 (352x1024)
    w_reg.append(temp_w_reg) #ext x pred1 x pred2 (352x1)

#     np.savez(wfm_save_dir+"/wfm_reg_epoch{}".format(epoch), X=w_reg)

    wfm_df = make_wfm(stats.zscore(w_reg[0].reshape(-1,)))

    cols = rows = wfm_df.columns.values
#     new_orig_network_idx_order = np.unique(cols)
    avg_df_1 = pd.DataFrame(columns=cols, index=rows)
    avg_df_2 = pd.DataFrame(columns=new_orig_network_idx_order, index=new_orig_network_idx_order)
    avg_df_3 = pd.DataFrame(columns=new_orig_network_idx_order, index=new_orig_network_idx_order)

    for i, temp_row in enumerate(new_orig_network_idx_order):
        for j, temp_col in enumerate(new_orig_network_idx_order):
            temp_row_ids = np.where(rows == temp_row)[0]
            temp_col_ids = np.where(cols == temp_col)[0]
            temp_mat = wfm_df.iloc[temp_row_ids, temp_col_ids]
            temp_vec = temp_mat.values.ravel()
            n_temp_vec = np.sum(np.absolute(temp_mat.values) > 1.96)
            if mode == 'sum':
                sum_val = temp_vec.sum()
                n_val = n_temp_vec
            elif mode == 'avg':
                if temp_row == temp_col:
                    sum_val = temp_vec.sum() / (len(temp_row_ids) * (len(temp_row_ids) - 1))
                    n_val = n_temp_vec / (len(temp_row_ids) * (len(temp_row_ids) - 1))
                else:
                    sum_val = temp_vec.sum() / (len(temp_row_ids) * (len(temp_col_ids)))
                    n_val = n_temp_vec / (len(temp_row_ids) * (len(temp_col_ids)))
            avg_df_1.iloc[temp_row_ids, temp_col_ids] = sum_val
            avg_df_2.loc[temp_row, temp_col] = sum_val
            avg_df_3.loc[temp_row, temp_col] = n_val

    if threshold==False:
        avg_mat = np.array(avg_df_2.values, dtype=np.float)
        avg_mat = (avg_mat - avg_mat.mean()) / avg_mat.std()
        avg_mat_df = pd.DataFrame(avg_mat, columns=new_orig_network_idx_order, index=new_orig_network_idx_order)
        avg_mat_df = avg_mat_df.sort_index(key=lambda x: x.map(new_orig_network_order), axis=0)
        avg_mat_df = avg_mat_df.sort_index(key=lambda x: x.map(new_orig_network_order), axis=1)
        thr_zero = 0
        avg_mat_df[(avg_mat_df < thr_zero) & (avg_mat_df > -thr_zero)] = 0

        sns.set(style="white", font_scale=4.5)
        fig, ax = plt.subplots(figsize=(32, 32))
        threshold = np.abs(avg_mat_df.values).max()
        threshold = 5.5
        cbar_kws = dict(
            use_gridspec=False, shrink=0.85, location="right", label="Correlation ($r$)")
        sns.heatmap(
            avg_mat_df, square=True, cmap="vlag", ax=ax, 
            mask=np.triu(np.ones(avg_mat_df.shape), 1).astype(np.bool),
            vmax=threshold, vmin=-threshold, 
            linewidths=5, 
            cbar=True, cbar_kws=cbar_kws,
            fmt=".2f", annot=True, annot_kws={"fontsize": 32}
        )
        cbar = ax.collections[0].colorbar
        cbar.set_label("$WF$", fontsize=60, labelpad=50)
        ax.set_xticklabels(new_orig_network_order, rotation=90, fontsize=60)
        ax.set_yticklabels(new_orig_network_order, rotation=0, fontsize=60)
        plt.title(f"Epoch {epoch}/{epochs}")
        plt.savefig(wfm_save_dir+f"/{mode}_WFM_epoch{epoch}.jpg")
        plt.close(fig)
    #     plt.show()

    if threshold==True:
        avg_mat = np.array(avg_df_2.values, dtype=np.float)
        avg_mat = (avg_mat - avg_mat.mean()) / avg_mat.std()
        avg_mat_df = pd.DataFrame(avg_mat, columns=new_orig_network_idx_order, index=new_orig_network_idx_order)
        avg_mat_df = avg_mat_df.sort_index(key=lambda x: x.map(new_orig_network_order), axis=0)
        avg_mat_df = avg_mat_df.sort_index(key=lambda x: x.map(new_orig_network_order), axis=1)

        annot = np.vectorize(lambda x: '' if np.absolute(x) < 3.091 else str(round(x, 2)))(avg_mat_df.to_numpy())
        # annot = np.vectorize(lambda x: '' if np.absolute(x) < 2.58 else str(round(x, 2)))(avg_mat_df.to_numpy())
        # annot = np.vectorize(lambda x: '' if np.absolute(x) < 1.96 else str(round(x, 2)))(avg_mat_df.to_numpy())

        thr_zero = 0
        avg_mat_df[(avg_mat_df < thr_zero) & (avg_mat_df > -thr_zero)] = 0

        sns.set(style="white", font_scale=5)
        plt.rcParams['mathtext.fontset'] = 'custom'
        plt.rcParams['mathtext.it'] = 'Arial:italic'
        plt.rcParams['mathtext.rm'] = 'Arial'

        fig, ax = plt.subplots(figsize=(32, 32))
        threshold = np.abs(avg_mat_df.values).max()
        threshold = 7
        cbar_kws = dict(
            use_gridspec=False, shrink=0.85, location="right", label="Correlation ($r$)")
        sns.heatmap(
            avg_mat_df, square=True, cmap="vlag",
            ax=ax,
            mask=np.triu(np.ones(avg_mat_df.shape), 1).astype(np.bool),
            vmax=threshold, vmin=-threshold, 
            linewidths=5, 
            cbar=True, cbar_kws=cbar_kws,
            fmt="", annot=annot, annot_kws={"fontsize": 36}
        )
        cbar = ax.collections[0].colorbar
        cbar.set_label("$WF$", fontsize=60, labelpad=50)
        ax.set_xticklabels(new_orig_network_order, rotation=90, fontsize=60)
        ax.set_yticklabels(new_orig_network_order, rotation=0, fontsize=60)
        plt.title(f"Epoch {epoch}/{epochs}")
        plt.savefig(wfm_save_dir+f"/{mode}_WFM_epoch{epoch}.jpg")
        plt.close(fig)
    #     plt.show()
    
    return

In [20]:
def plot_prediction_result(epoch,pearsonr,valid_prediction,valid_true):
    
    pred_save_dir = outer_save_dir+'/Prediction/'
    if not os.path.isdir(pred_save_dir):
        os.mkdir(pred_save_dir)
    
    sns.set(style="darkgrid", font_scale=2)
    plt.figure(figsize=(10,7))
    fig = sns.scatterplot(valid_true, valid_prediction,s=100)
    plt.title(f"Epoch {epoch}/{epochs}, Pearson's r : %.3f" % pearsonr)
    plt.xlabel("True $p$-factor")
    plt.ylabel("Predicted $p$-factor")
    plt.tight_layout()
    plt.ylim(np.min(valid_prediction)-0.0002,np.max(valid_prediction)+0.0002)
    plt.savefig(pred_save_dir+f"/Prediction_epoch{epoch}.png")
    plt.close()
    return 

## 4. Make dataset & Build model

In [21]:
#Hyperparameters for training
mode = "max"
min_lr = 1e-08
lr_alpha = -1.5
lr_beta = 1.7

momentum = 0.90
l1_param = 0
early_stopping_patience = 150

input_dim = X.shape[1]
n_classes = 1
output_dim = n_classes

#Hyperparameter for sparsity control
wsc_flag = [1, 1]
beta_lr = [5e-03, 1e-02]
max_beta = [5e-02, 1e-01]
n_wsc = wsc_flag.count(1)

In [22]:
# Make dataset
class TrainDataset(Dataset): 
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def __len__(self):
        return len(self.X_train)
    
    def __getitem__(self, idx): 
        X_train = torch.from_numpy(self.X_train[idx]).type(torch.FloatTensor)
        y_train = torch.from_numpy(self.y_train[idx]).type(torch.FloatTensor)

        return X_train, y_train

In [23]:
# Test dataset
class TestDataset(Dataset): 
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        
    def __len__(self):
        return len(self.X_test)
    
    def __getitem__(self, idx): 
        X_test = torch.from_numpy(self.X_test[idx]).type(torch.FloatTensor)
        y_test = torch.from_numpy(self.y_test[idx]).type(torch.FloatTensor)
        
        return X_test, y_test

In [24]:
#Build model
class DNN(nn.Module):
    def __init__(self, h1, h2, dropout_h1, dropout_h2, act_func_name):
        super(DNN, self).__init__()
        self.ext_1 = nn.Linear(input_dim, h1)
        self.ext_bn_1 = nn.BatchNorm1d(h1)
        
        self.prd_1 = nn.Linear(h1, h2)
        self.prd_bn_1 = nn.BatchNorm1d(h2)
        self.prd_2 = nn.Linear(h2, output_dim)
        
        self.dropout_h1 = nn.Dropout(p=dropout_h1)
        self.dropout_h2 = nn.Dropout(p=dropout_h2)
        
        self.act_func = get_act_func(act_func_name)
        self.weights_init()
    
    def forward(self, x):
        x = self.ext_1(x)
        x = self.ext_bn_1(x)
        x = self.act_func(x)
        x = self.dropout_h1(x)
        
        x = self.prd_1(x)
        x = self.prd_bn_1(x)
        x = self.act_func(x)
        x = self.dropout_h2(x)
        out = self.prd_2(x)
        
        return out
    
    def weights_init(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode="fan_in", nonlinearity="relu")
                nn.init.normal_(m.bias, std=0.01)

In [25]:
def get_optimizer(model, opt_name, learning_rate=None, l2_param=None):
    lower_opt_name = opt_name.lower()
    if lower_opt_name == 'momentum':
        return optim.SGD(model.parameters(), lr=learning_rate, 
                         momentum=momentum, weight_decay=l2_param)
    elif lower_opt_name == 'nag':
        return optim.SGD(model.parameters(), lr=learning_rate, 
                         momentum=momentum, weight_decay=l2_param, nesterov=True)
    elif lower_opt_name == 'adam':
        return optim.Adam(model.parameters(), lr=learning_rate, 
                          weight_decay=l2_param)
    elif lower_opt_name == 'sparseadam':
        return optim.SparseAdam(model.parameters(), lr=learning_rate,
                       betas=(0.9, 0.999), eps=1e-08, maximize=False)
    elif lower_opt_name == 'radam':
        return optim.RAdam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999),
                           eps=1e-08, weight_decay=l2_param)
    elif lower_opt_name == 'nadam':
        return optim.NAdam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08,
                  weight_decay=l2_param, momentum_decay=0.004)
    elif lower_opt_name == 'adamax':
        return optim.Adamax(model.parameters(), lr=learning_rate, betas=(0.9, 0.999),
                            eps=1e-08, weight_decay=l2_param)
    else:
        sys.exit("Illegal arguement for optimizer type")

In [26]:
def get_act_func(act_func_name):
    act_func_name = act_func_name.lower()
    if act_func_name == 'relu':
        return nn.ReLU()
    elif act_func_name == 'prelu':
        return nn.PReLU()
    elif act_func_name == 'elu':
        return nn.ELU()
    elif act_func_name == 'silu':
        return nn.SiLU()
    elif act_func_name == 'leakyrelu':
        return nn.LeakyReLU()
    elif act_func_name == 'tanh':
        return nn.Tanh()
    elif act_func_name == 'selu':
        return nn.SELU()
    elif act_func_name == 'gelu':
        return nn.GELU()
    else:
        sys.exit("Illegal arguement for activation function type")

In [27]:
def init_hsp(n_wsc, epochs):
    hsp_val = torch.zeros(n_wsc)
    beta_val = torch.clone(hsp_val)
    hsp_list = torch.zeros((n_wsc, epochs))
    beta_list = torch.zeros((n_wsc, epochs))
    
    return hsp_val, beta_val, hsp_list, beta_list

In [28]:
# Weight sparsity control with Hoyer's sparsness (Layer wise)
def calc_hsp(w, beta, max_beta, beta_lr, tg_hsp):
    
    # Get value of weight
    [dim, n_nodes] = w.shape
    num_elements = dim * n_nodes
    norm_ratio = torch.norm(w.detach(), 1) / torch.norm(w.detach(), 2)

    # Calculate hoyer's sparsity level
    num = math.sqrt(num_elements) - norm_ratio
    den = math.sqrt(num_elements) - 1
    hsp = torch.tensor(num / den).to(device)

    # Update beta
    beta = beta.clone() + beta_lr * torch.sign(torch.tensor(tg_hsp).to(device) - hsp)
    
    # Trim value
    beta = 0 if beta < 0 else beta
    beta = max_beta if beta > max_beta else beta

    return [hsp, beta]

In [29]:
def calc_l1(model, epoch, hsp_val, beta_val, hsp_list, beta_list, tg_hsp):
    l1_reg = None
    layer_idx = 0
    wsc_idx = 0

    for name, param in model.named_parameters():
        if "weight" in name and "bn" not in name:
            if "ext" in name or "prd_1" in name:
                temp_w = param
                
                if wsc_flag[layer_idx] != 0:
                    hsp_val[wsc_idx], beta_val[wsc_idx] = calc_hsp(
                        temp_w, beta_val[wsc_idx], max_beta[wsc_idx], 
                        beta_lr[wsc_idx], tg_hsp[wsc_idx]
                    )
                    hsp_list[wsc_idx, epoch - 1] = hsp_val[wsc_idx]
                    beta_list[wsc_idx, epoch - 1] = beta_val[wsc_idx]
                    layer_reg = torch.norm(temp_w, 1) * beta_val[wsc_idx].clone()
                    wsc_idx += 1
                else:
                    layer_reg = torch.norm(temp_w, 1) * l1_param

                if l1_reg is None:
                    l1_reg = layer_reg
                else:
                    l1_reg = l1_reg + layer_reg
                layer_idx += 1
        
    return l1_reg

In [30]:
def calc_pearsonr(x, y):
    x_mean = torch.mean(x.detach())
    y_mean = torch.mean(y.detach())
    xx = x.sub(x_mean)
    yy = y.sub(y_mean)
    num = xx.dot(yy)
    den = torch.norm(xx, 2) * torch.norm(yy, 2)
    corr = num / den
    return corr.item()

In [31]:
def train(model, epoch, train_loader, optimizer, criterion, 
          hsp_val, beta_val, hsp_list, beta_list, tg_hsp):
    model.train()
    train_loss = 0
    train_acc = 0
    cost = 0
    total = 0
    y_train_true = []
    y_train_pred = []
    
    
    for batch_idx, (input, target) in enumerate(train_loader):
        optimizer.zero_grad()
        input, target = input.to(device), target.to(device)
        pred, target = model(input), target.view(-1, 1)
        l1_norm = calc_l1(model, epoch, hsp_val, beta_val, hsp_list, beta_list, tg_hsp)
        running_loss = criterion(pred, target)
        cost = running_loss + l1_norm.clone()
        cost.backward()
        optimizer.step()
        train_loss += running_loss.item()
        total += pred.size(0)
        
        true_batch = torch.flatten(target).detach().cpu().numpy()
        pred_batch = torch.flatten(pred).detach().cpu().numpy()
        y_train_true.extend(true_batch)
        y_train_pred.extend(pred_batch)
    
    train_loss = train_loss / (batch_idx+1)
    train_acc,train_p = stats.pearsonr(y_train_true, y_train_pred)
    train_mae = mean_absolute_error(y_train_true, y_train_pred)

    return train_loss, train_acc,train_p,train_mae

In [32]:
def test(model, epoch, test_loader, criterion):
    model.eval()
    test_loss = 0
    test_acc = 0
    total = 0
    y_test_true = []
    y_test_pred = []
    
    with torch.no_grad():
        for batch_idx, (input, target) in enumerate(test_loader):
            input, target = input.to(device), target.to(device)
            pred, target = model(input), target.view(-1, 1)
            running_loss = criterion(pred, target)
            test_loss += running_loss.item()
            total += pred.size(0)

            true_batch = torch.flatten(target).detach().cpu().numpy()
            pred_batch = torch.flatten(pred).detach().cpu().numpy()
            y_test_true.extend(true_batch)
            y_test_pred.extend(pred_batch)

        test_acc,test_p = stats.pearsonr(y_test_true, y_test_pred)
        plot_prediction_result(epoch,test_acc,y_test_pred,y_test_true)
        test_loss = test_loss / (batch_idx+1)
        test_mae = mean_absolute_error(y_test_true, y_test_pred)


    return test_loss, test_acc, test_p,test_mae

In [33]:
def plot_learning_curves(
    save_dir, epochs, train_loss, valid_loss, train_acc, valid_acc, 
    lr, plot_hsp_list, plot_beta_list, tg_hsp, val_flg):
    
    sns.set(style="darkgrid", font_scale=2)
    fig, ax = plt.subplots(2, 3, figsize=(20, 10))
    ax = ax.flat
    lw = 4
    last_epoch = epochs
    
    ax[0].plot(train_loss[:last_epoch], label='train loss', lw=lw, color="g")
    if val_flg:
        ax[0].plot(valid_loss[:last_epoch], label='valid loss', lw=lw, color="orange")
    else:
        ax[0].plot(valid_loss[:last_epoch], label='test loss', lw=lw, color="r")
    ax[0].legend()
    ax[0].set_title("Loss Plot", pad=10)

    ax[1].plot(train_acc[:last_epoch], label='train corr', lw=lw, color="g")
    if val_flg:
        ax[1].plot(valid_acc[:last_epoch], label='valid corr', lw=lw, color="orange")
    else:
        ax[1].plot(valid_acc[:last_epoch], label='test corr', lw=lw, color="r")
    ax[1].legend()
    ax[1].set_title("Correlation Plot", pad=10)

    plot_hsp_list, plot_beta_list = np.array(plot_hsp_list).T, np.array(plot_beta_list).T
    
    for idx, n_layer in enumerate(indices):
        ax[3].plot(plot_hsp_list[idx], label='layer{}'.format(n_layer), lw=lw)
        ax[4].plot(plot_beta_list[idx], 
                   label='layer{}'.format(n_layer), lw=lw)
        ax[3].legend(); ax[4].legend()
        ax[3].set_title("HSP plot [{:.3f}/{:.3f}]"
                        .format(plot_hsp_list[0, -1], tg_hsp[0][0]), pad=10)
        ax[4].set_title("Beta plot", pad=10)
    
    #Learning rate plot
    ax[5].plot(lr[:last_epoch],label='Learning rate',lw=lw,color='m')
    ax[5].set_title("Learning rate")
    
    fig.tight_layout()
    fig.savefig("{}/Learning_curves.png".format(save_dir))
    
    plt.close(fig)

In [34]:
def run_outer_fold(n_outer_cv=0, outer_save_dir=None, sel_tg_hsp=None):
    val_flg = 0
    outer_cv_list = []
    
    # Outer fold
    print("\n===================================", end=" ")
    print("Outer Fold [{}/{}]".format(n_outer_cv + 1, outer_n_splits), end=" ")
    print("===================================")
    
    outer_start_fold_time = time.time()
    outer_train_idx = outer_train_folds_idx[n_outer_cv]
    outer_test_idx = outer_test_folds_idx[n_outer_cv]

    X_train, y_train = X[outer_train_idx], y[outer_train_idx]
    X_test, y_test = X[outer_test_idx], y[outer_test_idx]
    
    X_train = stats.zscore(X_train, axis=1)
    X_test = stats.zscore(X_test, axis=1)
        
    outer_train_dataset = TrainDataset(X_train, y_train)
    outer_test_dataset = TestDataset(X_test, y_test)
    
    outer_train_loader = DataLoader(
        outer_train_dataset, batch_size=batch_size, pin_memory=True,
        shuffle=True, num_workers=num_workers, drop_last=True)
    outer_test_loader = DataLoader(
        outer_test_dataset, batch_size=len(y_test), pin_memory=True,
        shuffle=True, num_workers=num_workers, drop_last=True)
        
    # Assign model 
    model = DNN(h1, h2, dropout_h1, dropout_h2, act_func_name).to(device)
    if pretrain:
        model_dict = model.state_dict()
        model_dict.update(pretrained_model_dict)
        model.load_state_dict(pretrained_model_dict)

        for name, child in model.named_children():
            if "ext" in name:
                for param in child.parameters():
                    param.requires_grad = freeze_ext
            elif "prd_1" in name:
                for param in child.parameters():
                    param.requires_grad = freeze_prd
    else:
        pass
            
    visualize_wfm(model,1,epoch_gap = 1,threshold=True, mode= 'sum')
    visualize_wfm(model,1,epoch_gap = 1,threshold=True, mode= 'avg')
    
    
    optimizer = get_optimizer(model, optimizer_name, learning_rate, l2_param)
    lr_factor = lr_alpha * sel_tg_hsp[0][0] + lr_beta
    scheduler = ReduceLROnPlateau(
        optimizer, mode=mode, patience=lr_patience, min_lr=min_lr, factor=lr_factor
    )
    cosine_scheduler = CosineAnnealingWarmRestarts(optimizer,25,eta_min = min_lr)
    criterion = nn.MSELoss()
              
    # list to save learning parameters
    outer_train_loss = []
    outer_test_loss = []
    outer_train_acc = []
    outer_test_acc = []
    outer_lr = []
    outer_hsp_list = []
    outer_beta_list = []

    hsp_val, beta_val, hsp_list, beta_list = init_hsp(n_wsc, epochs)
    
    
    for epoch in range(1, epochs + 1):
        train_loss, train_acc,train_p, train_mae = train(
            model, epoch, outer_train_loader, optimizer, criterion, 
            hsp_val, beta_val, hsp_list, beta_list, sel_tg_hsp
        )
        test_loss, test_acc,test_p, test_mae = test(model, epoch, outer_test_loader, criterion)
        if freeze_ext and hsp_val[0] < temp_param['hsp_h1']:
            scheduler.step(hsp_val[0])
        if hsp_val[0] >= temp_param['hsp_h1'] and epoch>100:
            cosine_scheduler.step()
        lr = optimizer.param_groups[0]['lr']
        
        outer_train_loss.append(train_loss)
        outer_train_acc.append(train_acc)
        outer_test_loss.append(test_loss)
        outer_test_acc.append(test_acc)
        outer_lr.append(lr)
        outer_hsp_list.append(hsp_val.tolist())
        outer_beta_list.append(beta_val.tolist())

        if epoch % print_epoch == 0:
            print("\nEpoch [{:d}/{:d}]".format(epoch, epochs), end=" ")
            print("Train acc: {:.2f}, Test acc: {:.2f}, Train loss: {:.4f}, Test loss: {:.4f}, Current_lr: {:.8f}"
                  .format(train_acc, test_acc, train_loss, test_loss,lr))
            for i in range(len(wsc_flag)):
                if wsc_flag[i] != 0:
                    print("Layer {:d}: [{:.4f}/{:.4f}]".
                          format( i + 1, hsp_val[i], sel_tg_hsp[i][0]), end=" ")

            plot_learning_curves(
                outer_save_dir, epochs, outer_train_loss, outer_test_loss,  
                outer_train_acc, outer_test_acc, 
                outer_lr, outer_hsp_list, outer_beta_list, sel_tg_hsp, val_flg
            )

        
    torch.save(model.state_dict(), 
               outer_save_dir + "/model_fold_" + str(n_outer_cv + 1) + ".pt")
    
    torch.cuda.empty_cache()
    gc.collect()
    
    outer_cv_list.append([train_acc, test_acc,train_p,test_p,test_loss,train_mae,test_mae])
            
    outer_cv_df = pd.DataFrame(np.array(outer_cv_list), columns=["train_acc", "test_acc","train_corr_p",
                                                                 "test_corr_p","test_loss","train_mae",
                                                                 "test_mae"])
    outer_cv_df.to_csv("{}/outer_cv.csv".format(outer_save_dir))

    outer_tot_time = time.time() - outer_start_fold_time
    print("\nExecution Time for Fold: {:.2f} mins".format(outer_tot_time / 60))
    
    return train_acc, test_acc, outer_hsp_list

## 5. Settle parameters, output path & Train model

In [35]:
param_grid = list(ParameterGrid(param_cand))

temp_param = param_grid[temp_sel_idx]

h1 = temp_param["h1"]
h2 = temp_param["h2"]

dropout_h1 = temp_param["dropout_h1"]
dropout_h2 = temp_param["dropout_h2"]

pretrain = temp_param["pretrain"]
freeze_ext = temp_param["freeze_ext"]
freeze_prd = temp_param["freeze_prd"]

batch_size = temp_param["batch_size"]
learning_rate = temp_param["lr"]
epochs = temp_param["epochs"]
l2_param = temp_param["l2_param"]

lr_patience = temp_param["lr_patience"]
lr_factor = temp_param["lr_factor"]

In [36]:
model = DNN(h1, h2, dropout_h1, dropout_h2, act_func_name).to(device)
cv_dir = prt_dir
cv_list = [i for i in os.listdir(cv_dir) if "Outer_fold_Full" not in i and 'WFM' not in i and "_weights" not in i]
cv_list.sort()

model_list = []
new_k = []
new_v = []

for i, j in enumerate(cv_list):
    tmp_path = "{}/model_fold_{}.pt".format(j, i + 1)
    model_path = os.path.join(cv_dir, tmp_path)
    tmp_model = torch.load(model_path)
    v_list = []
    for k, v in tmp_model.items():
        if k in model.state_dict():
            v_list.append(v)
            if i == 0:
                new_k.append(k)
    model_list.append(v_list)

for j in range(len(v_list)):
    for l, v in enumerate(model_list):
        if l == 0:
            tmp = model_list[l][j]
        else:
            tmp = torch.add(tmp, model_list[l][j])
    tmp_v = tmp / len(model_list)
    new_v.append(tmp_v)
    
pretrained_model_dict = {new_k[i]: new_v[i] for i in range(len(v_list))}

In [37]:
prt_hsp = prt_dir.split("Hsp:")[1].split("_")[0]
save_folder = f"Hsp:[{temp_param['hsp_h1']*wsc_flag[0]},{temp_param['hsp_h2']*wsc_flag[1]}]_Maxb:{max_beta}_Betalr:{beta_lr}_LR:[{temp_param['lr']}]_L2:[{l2_param}]_Act:[{act_func_name}]_Opt:[{optimizer_name}]_DO:[{temp_param['dropout_h1']},{temp_param['dropout_h2']}]_epoch:[{epochs}]_pretrain:{pretrain}_trainext:{freeze_ext}_trainprd:{freeze_prd}_seed:{seed}_from{prt_hsp}"
save_path = f"/users/hjd/IG_my_study/SNUH/data/temp/trasfer/{target_name}"
output_folder = os.path.join(save_path, save_folder)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
elif len(os.listdir(output_folder))>0:
    for fold in [f'Outer_fold_{i+1}' for i in select_fold]:
        if fold in os.listdir(output_folder):
            print(output_folder)
            raise Exception("Result already exist. Check directory!")
            
print(output_folder)

/users/hjd/IG_my_study/SNUH/data/temp/trasfer/p/Hsp:[0.95,0.3]_Maxb:[0.05, 0.1]_Betalr:[0.005, 0.01]_LR:[1e-05]_L2:[0.2]_Act:[elu]_Opt:[nag]_DO:[0.3,0.3]_epoch:[50]_pretrain:True_trainext:False_trainprd:True_seed:19_from[0.95,0.3,0.3]


In [38]:
print_epoch = 5

In [39]:
code_start_time = time.time()

print(output_folder)

outer_cv = []

for n_outer_cv in outer_cv_part:
    if n_outer_cv not in select_fold:
        continue    
    print("\n===================================", end=" ")
    print("Outer Fold [{}/{}]".format(n_outer_cv + 1, outer_n_splits), end=" ")
    print("===================================")

    outer_save_dir = "{}/Outer_fold_{}".format(output_folder, n_outer_cv + 1)
    os.makedirs(outer_save_dir, exist_ok=True)

    inner_cv = []
    temp_inner_cv_acc = []
    sel_idx = temp_sel_idx
    sel_param = param_grid[sel_idx]
    sel_hsp = []
    print("Selected param:", end=" ")
    for x in sel_param:
        if "hsp" in x: 
            print("{}: {}".format(x, sel_param[x]), end=" ")
            sel_hsp.append(sel_param[x])
    
    # Outer Fold
    hsp_cand_1 = [sel_param["hsp_h1"]]
    hsp_cand_2 = [sel_param["hsp_h2"]]

    indices = [i + 1 for i, x in enumerate(wsc_flag) if x == 1]
    hsp_cand_list = list(itertools.product(hsp_cand_1, hsp_cand_2))
    hsp_cand_list = [list(i) for i in hsp_cand_list]
    hsp_cand = [hsp_cand_1, hsp_cand_2]
    sel_tg_hsp = hsp_cand

    outer_train_acc, outer_test_acc, outer_hsp_list = run_outer_fold(
        n_outer_cv, outer_save_dir, sel_tg_hsp
    )
    outer_cv.append([sel_hsp, outer_train_acc, outer_test_acc])
    
    print("\nOuter Fold [{}/{}]: train acc: {:.4f}, test acc: {:.4f}"
          .format(n_outer_cv + 1, outer_n_splits, outer_train_acc, outer_test_acc))

/users/hjd/IG_my_study/SNUH/data/temp/trasfer/p/Hsp:[0.95,0.3]_Maxb:[0.05, 0.1]_Betalr:[0.005, 0.01]_LR:[1e-05]_L2:[0.2]_Act:[elu]_Opt:[nag]_DO:[0.3,0.3]_epoch:[50]_pretrain:True_trainext:False_trainprd:True_seed:19_from[0.95,0.3,0.3]

=================================== Outer Fold [1/20] ===================================
Selected param: hsp_h1: 0.95 hsp_h2: 0.3 
=================================== Outer Fold [1/20] ===================================


findfont: Font family ['cursive'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.



Epoch [5/50] Train acc: 0.43, Test acc: 0.03, Train loss: 0.4628, Test loss: 0.4588, Current_lr: 0.00001000
Layer 1: [0.9599/0.9500] Layer 2: [0.3026/0.3000] 
Epoch [10/50] Train acc: 0.33, Test acc: -0.06, Train loss: 0.4393, Test loss: 0.4708, Current_lr: 0.00001000
Layer 1: [0.9599/0.9500] Layer 2: [0.3025/0.3000] 
Epoch [15/50] Train acc: 0.48, Test acc: -0.14, Train loss: 0.3868, Test loss: 0.4878, Current_lr: 0.00001000
Layer 1: [0.9599/0.9500] Layer 2: [0.3024/0.3000] 
Epoch [20/50] Train acc: 0.53, Test acc: -0.08, Train loss: 0.3728, Test loss: 0.4803, Current_lr: 0.00001000
Layer 1: [0.9599/0.9500] Layer 2: [0.3023/0.3000] 
Epoch [25/50] Train acc: 0.57, Test acc: -0.22, Train loss: 0.3489, Test loss: 0.5226, Current_lr: 0.00001000
Layer 1: [0.9599/0.9500] Layer 2: [0.3023/0.3000] 
Epoch [30/50] Train acc: 0.49, Test acc: -0.15, Train loss: 0.3742, Test loss: 0.5031, Current_lr: 0.00001000
Layer 1: [0.9599/0.9500] Layer 2: [0.3022/0.3000] 
Epoch [35/50] Train acc: 0.59, Test

In [40]:
code_tot_time = time.time() - code_start_time 
print("Execution Time for the training: {:.2f} hours".format(code_tot_time / 60 / 60))

Execution Time for the training: 0.10 hours


In [41]:
df = pd.DataFrame(outer_cv, columns=["hsp", "train", "test"])
train_avg = np.array([x for x in df["train"].values]).mean()
test_avg = np.array([x for x in df["test"].values]).mean()
print("Train: {:.4f}, Test: {:.4f}".format(train_avg, test_avg))

Train: 0.5774, Test: -0.0968


In [42]:
tmp_param_grid = list(ParameterGrid(param_cand))[temp_sel_idx]
for name in tmp_param_grid:
    print(name, tmp_param_grid[name])

batch_size 2
dropout_h1 0.3
dropout_h2 0.3
epochs 50
freeze_ext False
freeze_prd True
h1 1024
h2 1024
hsp_h1 0.95
hsp_h2 0.3
l2_param 0.2
lr 1e-05
lr_factor 0.5
lr_patience 5
pretrain True


In [43]:
df['lr'] = learning_rate
df['beta_lr'] = [beta_lr]*len(df)
df['max_beta'] = [max_beta]*len(df)
df['train_idx'] = [outer_train_folds_idx[i] for i in select_fold]
df['test_idx'] = [outer_test_folds_idx[i] for i in select_fold]
df['l2_param'] = l2_param
df['batch_size'] = batch_size
df['act_func'] = act_func_name
df['optimizer'] = optimizer_name
df['momentum'] = momentum
df['epoch'] = epochs
df['pretrain_model_path'] = prt_dir

df.to_csv(output_folder+f"/result_df_{np.array(select_fold)+1}.csv", sep='\t',index=None)